In [2]:
import json
import requests as rq
import os
import time
import pandas as pd

In [75]:
def make_species_location_query(record):
    return {
        "rq": {
            "kingdom": str(record.kingdom),
            "genus": str(record.genus),
            "specificepithet": str(record.specificepithet),
            "country": str(record.country),
            "stateprovince": str(record.stateprovince),
            "county": str(record.county)
            },
        "limit": 1,
        "offset": 0
    }

In [17]:
presences = pd.read_csv(open("processed/t1-1-presence.tsv"), sep="\t")

In [5]:
absences = pd.read_csv(open("processed/t1-1-absence.tsv"), sep="\t")

In [6]:
absences.iloc[0]

kingdom                   animalia
phylum                    annelida
family                  nereididae
genus                       Nereis
specificepithet          vexillosa
country              United States
stateprovince             Michigan
county             Keweenaw County
Name: 0, dtype: object

Go through presence and absence records and check which ones are matched with records in iDigBio. Hopefully all presences match, and a minimal number of absences match

In [49]:
record = absences.iloc[0]
query = make_species_location_query(record)
response = rq.post("http://search.idigbio.org/v2/search/records/", json=query)
response.json()

{'itemCount': 0,
 'lastModified': '1970-01-01T00:00:00.000Z',
 'items': [],
 'attribution': []}

In [69]:
response.json()

{'itemCount': 0,
 'lastModified': '1970-01-01T00:00:00.000Z',
 'items': [],
 'attribution': []}

In [94]:
n = len(absences)
responses = {}
for record in absences.itertuples():
    query = make_species_location_query(record)
    response = rq.post("http://search.idigbio.org/v2/search/records/", json=query)
    responses[record.Index] = response.json()["itemCount"]
    print(f"{record.Index}, {len(responses)} / {n}", end="\r", flush=True)

absence_matches = pd.Series(responses)
m = sum(absence_matches == 0)
print(f"Records missing location: {m}/{n} ({100*m/n:2.2f}%)")

Records missing location: 9178/10004 (91.74%)


In [99]:
absences["valid"] = absence_matches == 0
absences["valid"].to_csv(open("processed/t1-1-absence-valid.tsv", "w"), sep="\t")

In [91]:
n = len(absences)
responses = {}
for record in presences.itertuples():
    query = make_species_location_query(record)
    response = rq.post("http://search.idigbio.org/v2/search/records/", json=query)
    responses[record.Index] = response.json()["itemCount"]
    print(f"{record.Index}, {len(responses)} / {n}", end="\r", flush=True)

presence_matches = pd.Series(responses)
m = sum(presence_matches == 0)
print(f"Records missing location: {m}/{n} ({100*m/n:2.2f}%)")

Records missing location: 24/10004 (0.24%)


In [93]:
presence_matches.head()

0    21
1    72
2    42
3     4
4     6
dtype: int64